# Imports

In [1]:
import os
import numpy as np
import healpy as hp
from astropy.io import fits
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM
from astropy.coordinates import SkyCoord
import pandas
import gammalib
import minot
from PerseusGammaCalibration import perseus_model_library
cosmo = FlatLambdaCDM(70, 0.3, Ob0=0.0224/0.674**2)

# Main parameters 

In [2]:
Nside = 1024
Npt_per_decade = 30  # Set the precision

ne_gas_poly = True
Fx500_treshold = 1e-3*5e-12 # erg/s/cm2

X_crp = 0.01
eta_crp = 1.0
alpha_crp = 2.4

workdir = '/Users/adam/Project/CTA/Phys/Outputs/CTADC2022/'

# Input catalog

### ACCEPT catalog

In [3]:
#---------- Extract Accept info
accept = pandas.read_table('/Users/adam/Project/ExternalData/accept_main.tab', sep='\s+')
accept_name = []
accept_ra   = []
accept_dec  = []
accept_z    = []
accept_k0   = []
for icl in range(1, len(accept)):
    accept_coord_icl = SkyCoord(accept['RA'][icl]+accept['Dec'][icl], unit=(u.hourangle, u.deg), frame='icrs')
    accept_ra.append(accept_coord_icl.ra.to_value('deg'))
    accept_dec.append(accept_coord_icl.dec.to_value('deg'))
    accept_z.append(float(accept['z'][icl]))
    accept_k0.append(float(accept['K0'][icl]))
    accept_name.append(accept['#Name'][icl])
accept_name = np.array(accept_name)
accept_ra   = np.array(accept_ra)
accept_dec  = np.array(accept_dec)
accept_z    = np.array(accept_z)
accept_k0   = np.array(accept_k0)

#---------- Add info for important clusters not in ACCEPT
# https://www.aanda.org/articles/aa/pdf/2010/05/aa12377-09.pdf
accept_name = np.append(accept_name, 'A2634')
accept_ra = np.append(accept_ra, 354.57181275)
accept_dec = np.append(accept_dec, 27.02934221)
accept_z = np.append(accept_z, 0.031)
accept_k0 = np.append(accept_k0, 40.7)

# https://www.aanda.org/articles/aa/pdf/2010/05/aa12377-09.pdf
accept_name = np.append(accept_name, 'A1367')
accept_ra = np.append(accept_ra, 176.12366417)
accept_dec = np.append(accept_dec, 19.83883798)
accept_z = np.append(accept_z, 0.022)
accept_k0 = np.append(accept_k0, 296.5)

# https://www.aanda.org/articles/aa/pdf/2010/05/aa12377-09.pdf
accept_name = np.append(accept_name, 'A1656')
accept_ra = np.append(accept_ra, 194.95038236)
accept_dec = np.append(accept_dec, 27.97997795)
accept_z = np.append(accept_z, 0.023)
accept_k0 = np.append(accept_k0, 364.6) 

# https://www.aanda.org/articles/aa/pdf/2010/05/aa12377-09.pdf
accept_name = np.append(accept_name, 'NGC1399')
accept_ra = np.append(accept_ra, 54.6157681)
accept_dec = np.append(accept_dec, -35.45277218)
accept_z = np.append(accept_z, 0.005)
accept_k0 = np.append(accept_k0, 13.6)

# No entropy floor seen? https://academic.oup.com/mnras/article/469/2/1476/3738101
accept_name = np.append(accept_name, 'Virgo')
accept_ra = np.append(accept_ra, 186.63483418)
accept_dec = np.append(accept_dec, 12.7233333)
accept_z = np.append(accept_z, 0.0036)
accept_k0 = np.append(accept_k0, 0)

# https://academic.oup.com/mnras/article/326/1/2/1026382
accept_name = np.append(accept_name, '3C 129.1')
accept_ra = np.append(accept_ra, 72.515)
accept_dec = np.append(accept_dec, 45.024)
accept_z = np.append(accept_z, 0.022)
accept_k0 = np.append(accept_k0, np.nan) 

# clear CC cluster http://www.aspbooks.org/publications/329/183.pdf
accept_name = np.append(accept_name, 'RXC J1324.7-5736')
accept_ra = np.append(accept_ra, 201.18)
accept_dec = np.append(accept_dec, -57.614)
accept_z = np.append(accept_z, 0.019)
accept_k0 = np.append(accept_k0, 0)

# https://academic.oup.com/pasj/article/64/1/16/1432259 (merging + CC)
accept_name = np.append(accept_name, 'A3627')
accept_ra = np.append(accept_ra, 243.594)
accept_dec = np.append(accept_dec, -60.869)
accept_z = np.append(accept_z, 0.0157)
accept_k0 = np.append(accept_k0, np.nan) 

# https://arxiv.org/pdf/astro-ph/9606112.pdf
accept_name = np.append(accept_name, 'Triangulum Australis')
accept_ra = np.append(accept_ra, 249.567)
accept_dec = np.append(accept_dec, -64.347)
accept_z = np.append(accept_z, 0.0508)
accept_k0 = np.append(accept_k0, 1e4) 

### MCXC catalog

In [4]:
mcxc = fits.open('/Users/adam/Project/ExternalData/Planck/Clusters/MCXC_official.fits')
mcxc_data = mcxc[1].data

mcxc_ra    = []
mcxc_dec   = []
mcxc_z     = []
mcxc_m     = []
mcxc_lx    = []
for icl in range(len(mcxc_data)):
    mcxc_ra.append(mcxc_data[icl][10])
    mcxc_dec.append(mcxc_data[icl][11])
    mcxc_z.append(mcxc_data[icl][14])
    mcxc_lx.append(mcxc_data[icl][18])
    mcxc_m.append(mcxc_data[icl][19])
mcxc_ra    = np.array(mcxc_ra)
mcxc_dec   = np.array(mcxc_dec)
mcxc_z     = np.array(mcxc_z)
mcxc_m     = np.array(mcxc_m)
mcxc_lx    = np.array(mcxc_lx)

### Define the subsample: MCXC threshold on F_X500 (i.e. mainly REFLEX/NORAS/CIZA)

In [5]:
cat_name = []
cat_ra   = []
cat_dec  = []
cat_z    = []
cat_mass = []
cat_lx   = []
cat_K0   = []
cat_cat  = []

#---------- Start with MCXC and treshold it
no_k0_count = 0
for icl in range(len(mcxc[1].data)):
    Dl = cosmo.luminosity_distance(mcxc[1].data[icl][14]).to_value('cm')
    if 1e44*mcxc[1].data[icl][18]**1.2/(4*np.pi*Dl**2) > Fx500_treshold: 
        # Use baseline name unless empty
        if mcxc[1].data[icl][2] == '':
            cat_name.append(mcxc[1].data[icl][1].replace(" ", "_").replace("(", "").replace(")", ""))
        else:
            cat_name.append(mcxc[1].data[icl][2].replace(" ", "_").replace("(", "").replace(")", ""))
            
        cat_ra.append(mcxc_data[icl][10])
        cat_dec.append(mcxc_data[icl][11])
        cat_z.append(mcxc_data[icl][14])
        cat_mass.append(mcxc_data[icl][19])
        cat_lx.append(mcxc_data[icl][18])
        cat_cat.append(mcxc_data[icl][16])

        #---------- Match to ACCEPT to get entropy
        dist = minot.ClusterTools.map_tools.greatcircle(accept_ra, accept_dec, 
                                                        mcxc[1].data[icl][10], mcxc_data[icl][11])
        c1 = (dist < 15.0/60)
        c2 = np.abs(accept_z - mcxc_data[icl][14]) < 0.1
        w    = c1*c2
        if np.sum(w) == 1: 
            cat_K0.append(accept_k0[w][0])
        else:
            print('     WARNING: Nmatch=', np.sum(w), 
                  1e44*mcxc[1].data[icl][18]**1.2/(4*np.pi*Dl**2)/Fx500_treshold, 
                  mcxc_data[icl][10], mcxc_data[icl][11], mcxc_data[icl][14],
                  mcxc[1].data[icl][1], mcxc[1].data[icl][2])
            cat_K0.append(np.nan)
            no_k0_count+=1
            
print('No K0 for '+str(no_k0_count)+'/'+str(len(cat_name)))

cat_name = np.array(cat_name)
cat_ra   = np.array(cat_ra)
cat_dec  = np.array(cat_dec)
cat_z    = np.array(cat_z)
cat_mass = np.array(cat_mass)
cat_lx   = np.array(cat_lx)
cat_K0   = np.array(cat_K0)
cat_cat  = np.array(cat_cat)

No K0 for 1506/1743


In [6]:
catalog = {'name': cat_name, 
           'RA': cat_ra, 'Dec':cat_dec, 
           'redshift':cat_z, 'Mass':cat_mass, 'Lx':cat_lx,
           'K0':cat_K0, 'catalog':cat_cat}
df = pandas.DataFrame(data=catalog)
df.to_csv('/Users/adam/Desktop/catalog.csv')
df

,name,RA,Dec,redshift,Mass,Lx,K0,catalog
0,UGC_12890,0.030,8.274,0.0396,0.7373,0.196280,NaN,NORAS
1,RXC_J0000.4-0237,0.103,-2.625,0.0379,0.3297,0.052338,NaN,SGP
2,RXC_J0001.6-1540,0.412,-15.681,0.1246,1.6557,0.814902,NaN,SGP
3,A2692,0.488,12.073,0.2033,2.6927,1.990205,NaN,NORAS
4,A2697,0.799,-6.086,0.2320,5.2188,6.107094,NaN,REFLEX
...,...,...,...,...,...,...,...,...
1738,A4059,359.260,-34.761,0.0475,2.6658,1.629622,7.06,REFLEX
1739,A4067,359.830,-60.700,0.0989,1.9699,1.051539,NaN,REFLEX
1740,MS2356.9-3434,359.864,-34.306,0.1150,0.7568,0.223131,NaN,EMSS_1994
1741,BSe_RXJ2359.5-3211,359.900,-32.185,0.4780,1.4784,1.069996,NaN,SHARC_SOUTH


In [7]:
from astropy.table import Table
t = Table.from_pandas(df)
t.write('/Users/adam/Desktop/catalog.fits', overwrite=True)
new_table = Table.read('/Users/adam/Desktop/catalog.fits')
new_table

name,RA,Dec,redshift,Mass,Lx,K0,catalog
bytes50,float64,float64,float64,float64,float64,float64,bytes12
UGC_12890,0.03,8.274,0.0396,0.7373,0.19628,1e+20,NORAS
RXC_J0000.4-0237,0.103,-2.625,0.0379,0.3297,0.052338,1e+20,SGP
RXC_J0001.6-1540,0.412,-15.681,0.1246,1.6557,0.814902,1e+20,SGP
A2692,0.488,12.073,0.2033,2.6927,1.990205,1e+20,NORAS
A2697,0.799,-6.086,0.232,5.2188,6.107094,1e+20,REFLEX
A2717,0.801,-35.927,0.049,1.2016,0.441862,27.05,REFLEX
A2700,0.961,2.063,0.0924,1.7342,0.846881,1e+20,REFLEX
UGC_00032,1.247,11.701,0.0761,1.3009,0.518795,1e+20,NORAS
A2703,1.344,16.211,0.1164,2.4929,1.579007,1e+20,NORAS


# Compute sky maps looping over catalog

In [6]:
#---------- Energy bining definition
Ebin_emin = 100*u.GeV
Ebin_emax = 100*u.TeV
Ebin_nbin = 3

Ebin_pot    = np.logspace(np.log10(Ebin_emin.to_value('GeV')), 
                          np.log10(Ebin_emax.to_value('GeV')), 
                          Ebin_nbin+1)*u.GeV
Ebin_ctr    = ((Ebin_pot + np.roll(Ebin_pot,1))/2)[1:]
Ebin_logctr = 10**((np.log10(Ebin_pot.to_value('GeV')) + np.roll(np.log10(Ebin_pot.to_value('GeV')),1))/2)[1:]

Ebin_pot, Ebin_ctr, Ebin_logctr

(<Quantity [   100.,   1000.,  10000., 100000.] GeV>,
 <Quantity [  550.,  5500., 55000.] GeV>,
 array([  316.22776602,  3162.27766017, 31622.77660168]))

In [7]:
#---------- Helpix bining
skymap = np.zeros((Ebin_nbin, hp.nside2npix(Nside)))

In [8]:
#---------- Init the healpix to get lon and lat maps
bid_cl = minot.Cluster(silent=True)
bid_map, lon, lat = bid_cl.get_gamma_hpmap(nside=Nside, output_lonlat=True)

/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/healpy/pixelfunc.py:528: RuntimeWarning: divide by zero encountered in _pix2ang_ring
  theta, phi = pixlib._pix2ang_ring(nside, ipix)
/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/healpy/pixelfunc.py:528: RuntimeWarning: invalid value encountered in _pix2ang_ring
  theta, phi = pixlib._pix2ang_ring(nside, ipix)
/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/minot/ClusterTools/map_tools.py:216: RuntimeWarning: invalid value encountered in sin
  arg4 = (np.sin((lon_ref-lon)*np.pi/180.0/2.0))**2


In [9]:
K0_limits = [25, 70] # Used to define cool-cores, morphologically disturbed or universal clusters

TemplateDir = workdir+'Template_X_'+str(X_crp)+'_alpha_'+str(alpha_crp)+'_eta'+str(eta_crp)
if not os.path.exists(TemplateDir): os.mkdir(TemplateDir)
model_tot = gammalib.GModels()

for icl in range(len(cat_name)):
    print(str(icl+1)+'/'+str(len(cat_name)), cat_name[icl], cat_ra[icl], cat_dec[icl], cat_mass[icl], cat_z[icl])
    
    #---------- Special case for Perseus
    if cat_name[icl] == 'A0426': 
        print('   --> This is Perseus!')
        cluster = perseus_model_library.default_model()
        cluster.name = 'A0426'
        cluster.output_dir = workdir
        cluster.Npt_per_decade_integ = Npt_per_decade
        cluster.R_truncation = 3*cluster.R500
        cluster.map_coord = cluster.coord
        cluster.map_reso = 0.01*u.deg
        cluster.map_fov = 2.5*cluster.theta_truncation

    #---------- Otherwise build a standard model
    elif cat_mass[icl]/cat_mass[icl] == 1:
        # Init the model
        cluster = minot.Cluster(name=cat_name[icl], M500=cat_mass[icl]*1e14*u.Msun, redshift=cat_z[icl], 
                                RA=cat_ra[icl]*u.deg, Dec=cat_dec[icl]*u.deg, 
                                cosmology=cosmo, silent=True, output_dir=workdir)
        cluster.Npt_per_decade_integ = Npt_per_decade
        cluster.R_truncation = 3*cluster.R500
        cluster.map_coord = SkyCoord(cat_ra[icl]*u.deg, cat_dec[icl]*u.deg, frame='icrs')
        cluster.map_reso = 0.01*u.deg
        cluster.map_fov = 2.5*cluster.theta_truncation
        
        # Select the thermodynamic profile using K0
        if cat_K0[icl] is np.nan:                                            # Undefined --> UPP
            cluster.set_pressure_gas_gNFW_param(pressure_model='A10UPP')
            cluster.set_density_gas_universal_param(density_model='G19UDP')
        else:
            if cat_K0[icl] < K0_limits[0]:                                   # Low K0 --> CC
                cluster.set_pressure_gas_gNFW_param(pressure_model='A10CC')
                cluster.set_density_gas_universal_param(density_model='G19CC')
            elif cat_K0[icl] >= K0_limits[0] and cat_K0[icl] < K0_limits[1]: # Intermediate K0 --> CC
                cluster.set_pressure_gas_gNFW_param(pressure_model='A10UPP')
                cluster.set_density_gas_universal_param(density_model='G19UDP')
            else:                                                            # High K0 --> MD
                cluster.set_pressure_gas_gNFW_param(pressure_model='A10MD')
                cluster.set_density_gas_universal_param(density_model='G19MD')
        
        # Try polytropic expectation for density
        if ne_gas_poly:
            rad, prof = cluster.get_pressure_gas_profile(radius=np.logspace(-1,4,10000)*u.kpc)
            E_z   = cluster.cosmo.efunc(cluster.redshift)
            h70   = cluster.cosmo.H0.value/70.0
            mu_g,mu_e,mu_p,mu_a = minot.ClusterTools.cluster_global.mean_molecular_weight(Y=cluster.helium_mass_fraction,
                                                                                          Z=cluster.metallicity_sol*cluster.abundance)
            P500 = 3.426*1e-3 * (cluster.M500.to_value('Msun')*h70/1e15)**(2.0/3) * E_z**(8.0/3) * ((cluster.cosmo.Ob0/cluster.cosmo.Om0)/0.16)*(mu_g/0.6)*(mu_e/1.14)
            P0 = np.exp(-2.94)
            n0 = np.exp(-10.3)
            Gamma = 1.19
            profile = (prof.to_value('keV cm-3')/(P500*P0))**(1/Gamma) * n0*E_z**2 * u.cm**-3
            cluster.density_gas_model = {'name':'User', 'radius':rad, 'profile':profile}
        
        # Set the CR
        cluster.X_cre1_E = {'X':0,    'R_norm':cluster.R500}
        cluster.X_crp_E  = {'X':X_crp, 'R_norm':cluster.R500}
        cluster.set_density_crp_isodens_scal_param(scal=eta_crp)
        cluster.spectrum_crp_model = {'name':'PowerLaw', 'Index': alpha_crp}
    
    #---------- Compute the sky map in each energy bin and sum
    for ieb in range(Ebin_nbin):
        map_ebin = cluster.get_gamma_hpmap(nside=Nside, 
                                           Emin=Ebin_pot[ieb], Emax=Ebin_pot[ieb+1], 
                                           Rmin_los=1*u.kpc, NR500_los=3.0, Rmin=1*u.kpc, Rmax=3*cluster.R500, 
                                           Energy_density=False, Cframe=False, model='Kafexhiu2014', 
                                           maplonlat=[lon,lat], output_lonlat=False)
        skymap[ieb,:] += map_ebin.to_value('s-1 sr-1 cm-2')
            
    #---------- Compute the templates
    #----- Map
    header = cluster.get_map_header()
    image = cluster.get_gamma_map(Emin=5e-2*u.TeV, Emax=1e+2*u.TeV, # Emin/Emax not important since normalized
                                  Rmin_los=None, NR500_los=5.0, Rmin=None, Rmax=None,
                                  Normalize=True)
    hdu = fits.PrimaryHDU(header=header)
    hdu.data = image.value
    hdu.header.add_comment('Gamma-ray normalized map')
    hdu.header.add_comment('Unit = '+str(image.unit))
    hdu.writeto(TemplateDir+'/TemplateMap_'+cluster.name+'.fits', overwrite=True)

    #----- Spectrum
    energy = np.logspace(-2,6,1000)*u.GeV
    eng, spec = cluster.get_gamma_spectrum(energy,
                                           Rmin=None, Rmax=cluster.R_truncation, Rmin_los=None, NR500_los=5.0,
                                           type_integral='spherical')
    wgood  = spec > 0
    energy = energy[wgood]
    spec   = spec[wgood]
    cluster._save_txt_file(TemplateDir+'/TemplateSpectrum_'+cluster.name+'.txt', 
                           energy.to_value('MeV'), spec.to_value('MeV-1 cm-2 s-1'),
                           'energy (MeV)', 'spectrum (MeV-1 cm-2 s-1)')
    
    #----- Add model
    spatial  = gammalib.GModelSpatialDiffuseMap(TemplateDir+'/TemplateMap_'+cluster.name+'.fits')
    spectral = gammalib.GModelSpectralFunc(TemplateDir+'/TemplateSpectrum_'+cluster.name+'.txt', 1.0)
    timing   = gammalib.GModelTemporalConst(1)
    model    = gammalib.GModelSky(spatial, spectral, timing)
    model.name(cluster.name)
    model.tscalc(True)
    model_tot.append(model)
    
model_tot.save(TemplateDir+'/ClusterTemplates.xml')

1/484 A2697 0.799 -6.086 5.2188 0.232


/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/minot/ClusterTools/map_tools.py:216: RuntimeWarning: invalid value encountered in sin
  arg4 = (np.sin((lon_ref-lon)*np.pi/180.0/2.0))**2
/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/minot/ClusterTools/map_tools.py:216: RuntimeWarning: invalid value encountered in sin
  arg4 = (np.sin((lon_ref-lon)*np.pi/180.0/2.0))**2
/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/minot/ClusterTools/map_tools.py:216: RuntimeWarning: invalid value encountered in sin
  arg4 = (np.sin((lon_ref-lon)*np.pi/180.0/2.0))**2


2/484 A2717 0.801 -35.927 1.2016 0.049


/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/minot/ClusterTools/map_tools.py:216: RuntimeWarning: invalid value encountered in sin
  arg4 = (np.sin((lon_ref-lon)*np.pi/180.0/2.0))**2
/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/minot/ClusterTools/map_tools.py:216: RuntimeWarning: invalid value encountered in sin
  arg4 = (np.sin((lon_ref-lon)*np.pi/180.0/2.0))**2
/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/minot/ClusterTools/map_tools.py:216: RuntimeWarning: invalid value encountered in sin
  arg4 = (np.sin((lon_ref-lon)*np.pi/180.0/2.0))**2


3/484 A2721 1.513 -34.724 2.7122 0.1147


...

/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/minot/ClusterTools/map_tools.py:216: RuntimeWarning: invalid value encountered in sin
  arg4 = (np.sin((lon_ref-lon)*np.pi/180.0/2.0))**2
/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/minot/ClusterTools/map_tools.py:216: RuntimeWarning: invalid value encountered in sin
  arg4 = (np.sin((lon_ref-lon)*np.pi/180.0/2.0))**2
/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/minot/ClusterTools/map_tools.py:216: RuntimeWarning: invalid value encountered in sin
  arg4 = (np.sin((lon_ref-lon)*np.pi/180.0/2.0))**2


483/484 A2675 358.909 11.343 1.6934 0.0726


/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/minot/ClusterTools/map_tools.py:216: RuntimeWarning: invalid value encountered in sin
  arg4 = (np.sin((lon_ref-lon)*np.pi/180.0/2.0))**2
/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/minot/ClusterTools/map_tools.py:216: RuntimeWarning: invalid value encountered in sin
  arg4 = (np.sin((lon_ref-lon)*np.pi/180.0/2.0))**2
/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/minot/ClusterTools/map_tools.py:216: RuntimeWarning: invalid value encountered in sin
  arg4 = (np.sin((lon_ref-lon)*np.pi/180.0/2.0))**2


484/484 A4059 359.26 -34.761 2.6658 0.0475


/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/minot/ClusterTools/map_tools.py:216: RuntimeWarning: invalid value encountered in sin
  arg4 = (np.sin((lon_ref-lon)*np.pi/180.0/2.0))**2
/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/minot/ClusterTools/map_tools.py:216: RuntimeWarning: invalid value encountered in sin
  arg4 = (np.sin((lon_ref-lon)*np.pi/180.0/2.0))**2
/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/minot/ClusterTools/map_tools.py:216: RuntimeWarning: invalid value encountered in sin
  arg4 = (np.sin((lon_ref-lon)*np.pi/180.0/2.0))**2


In [10]:
#---------- Save the maps as fits files
filename = workdir+'cluster_sky_map_Vdc.fits'

column_names = []
for ieb in range(Ebin_nbin):
    column_names.append('Gamma-ray in E=['+str(Ebin_pot[ieb].to_value('GeV'))+','+str(Ebin_pot[ieb+1].to_value('GeV'))+'] GeV')

hp.write_map(filename, skymap, overwrite=True, fits_IDL=False,
             column_names=column_names,
             column_units='s-1 sr-1 cm-2')

Ebin_hdr = fits.Header()
Ebin_hdr.add_comment('Energy bin edges (GeV)')
fits.append(filename, Ebin_pot.to_value('GeV'), Ebin_hdr)